In [ ]:
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re
import os
import tiktoken

# Pasta com PDFs
pdf_folder = r"D:\Ciencia de Dados\Projeto Integrador de Extenção 3\RAGzilla\project\data"

# Lista todos os PDFs
pdf_files = [f for f in os.listdir(pdf_folder) if f.lower().endswith(".pdf")]

# Função de limpeza de texto
def clean_text(text):
    text = re.sub(r'[^\w\s.,;:!?()-]', '', text)
    text = re.sub('__', '', text)
    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

encoding = tiktoken.get_encoding("cl100k_base")

def text_to_tokens(text):
    return encoding.encode(text)

def tokens_to_text(tokens):
    return encoding.decode(tokens)

# Split por tokens
def chunk_text_by_tokens(text, max_tokens, overlap):
    tokens = text_to_tokens(text)
    chunks = []
    start = 0
    while start < len(tokens):
        end = min(start + max_tokens, len(tokens))
        chunk_tokens = tokens[start:end]
        chunk_text = tokens_to_text(chunk_tokens)
        chunks.append(chunk_text)
        start += max_tokens - overlap  # move com overlap
    return chunks

# Lista final de chunks com metadados
all_chunks = []

for pdf_file in pdf_files:
    pdf_path = os.path.join(pdf_folder, pdf_file)
    reader = PdfReader(pdf_path)

    for i, page in enumerate(reader.pages, start=1):
        page_text = page.extract_text()
        if page_text:
            page_text = clean_text(page_text)
            chunks = chunk_text_by_tokens(page_text, max_tokens=1000, overlap=50)
            
            for chunk in chunks:
                all_chunks.append({
                    "text": chunk,
                    "source_file": pdf_file,
                    "page_number": i
                })

print(f"Total de chunks gerados: {len(all_chunks)}")
print("Exemplo de chunk com metadados:\n", all_chunks[0])


Total de chunks gerados: 117
Exemplo de chunk com metadados:
 {'text': 'Página 1 de 12 CONTRATO DE ASSOCIAÇÃO DE EMPRESA Pelo presente instrumento e na melhor forma de direito, de um lado, AJJ SERVIÇOS CONTABÉIS LTDA, pessoa jurídica de direito privado, inscrita no CNPJ sob o nº 28. 913.7060001-63, com sede na Rua Alfred Charvet, nº 710, Bairro Vila Nova, AraucáriaPR, CEP: 83.703-278, neste ato, representada por seu sócio administrador, Jhonny Cézar de Jesus Falavinha, brasileiro, portador do RG nº 6.223.402-4 SSPPR, inscrito no CPF nº 007.724.809-07, doravante denominada EMPRESA ASSOCIADA, e de outro lado, o PARQUE CIENTÍFICO E TECNOLÓGICO DE BIOCIÊNCIAS LTDA, pessoa jurídica de direito privado, inscrito no CNPJ sob o nº 21.526.7090001-03, com sede na Rodovia PR-182, Km 320321, sn, Condomínio Industrial Biopark, no município de Toledo, Estado do Paraná, CEP 85.919-899, neste ato representada por seu sócio administrador, Victor Donaduzzi, brasileiro, inscrito no CPF sob o nº 047.139.43

In [5]:
import json
import os

# Pasta de destino
output_folder = r"D:\Ciencia de Dados\Projeto Integrador de Extenção 3\RAGzilla\project\data\processed"
os.makedirs(output_folder, exist_ok=True)

# Nome do arquivo de saída
output_file = os.path.join(output_folder, "chunks.json")

# Salvar todos os chunks
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(all_chunks, f, ensure_ascii=False, indent=4)

print(f"✅ {len(all_chunks)} chunks salvos em {output_file}")

✅ 117 chunks salvos em D:\Ciencia de Dados\Projeto Integrador de Extenção 3\RAGzilla\project\data\processed\chunks.json


In [1]:
import json

input_file = r"D:\Ciencia de Dados\Projeto Integrador de Extenção 3\RAGzilla\project\data\processed\chunks.json"

with open(input_file, "r", encoding="utf-8") as f:
    chunks = json.load(f)

print(f"Total de chunks carregados: {len(chunks)}")
print("Exemplo:", chunks[0])

Total de chunks carregados: 117
Exemplo: {'text': 'Página 1 de 12 CONTRATO DE ASSOCIAÇÃO DE EMPRESA Pelo presente instrumento e na melhor forma de direito, de um lado, AJJ SERVIÇOS CONTABÉIS LTDA, pessoa jurídica de direito privado, inscrita no CNPJ sob o nº 28. 913.7060001-63, com sede na Rua Alfred Charvet, nº 710, Bairro Vila Nova, AraucáriaPR, CEP: 83.703-278, neste ato, representada por seu sócio administrador, Jhonny Cézar de Jesus Falavinha, brasileiro, portador do RG nº 6.223.402-4 SSPPR, inscrito no CPF nº 007.724.809-07, doravante denominada EMPRESA ASSOCIADA, e de outro lado, o PARQUE CIENTÍFICO E TECNOLÓGICO DE BIOCIÊNCIAS LTDA, pessoa jurídica de direito privado, inscrito no CNPJ sob o nº 21.526.7090001-03, com sede na Rodovia PR-182, Km 320321, sn, Condomínio Industrial Biopark, no município de Toledo, Estado do Paraná, CEP 85.919-899, neste ato representada por seu sócio administrador, Victor Donaduzzi, brasileiro, inscrito no CPF sob o nº 047.139.439-40, doravante denom

In [2]:
from openai import OpenAI

from dotenv import load_dotenv
import os

# Carregar variáveis do .env
load_dotenv()

# OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

embeddings_list = []

for chunk in chunks:
    response = client.embeddings.create(
        model="text-embedding-3-small",  # ou text-embedding-3-large
        input=chunk["text"]
    )
    emb = response.data[0].embedding
    embeddings_list.append({
        "embedding": emb,
        "source_file": chunk["source_file"],
        "page_number": chunk["page_number"]
    })

print("✅ Embeddings gerados para todos os chunks!")

✅ Embeddings gerados para todos os chunks!


In [ ]:
###DAQUI PARA BAIXO IGNORA

In [ ]:
def rag_manual_chat(query, openai_client, index, threshold=0.1, top_k=3):
    
    # Gerar embedding da query
    query_embedding = openai_client.embeddings.create(
        input=query,
        model="text-embedding-3-small"
    ).data[0].embedding

    # Filtrar pelo threshold
    matches = [m for m in response.matches if m.score >= threshold]

    if not matches:
        return "Desculpe, não encontrei informações relevantes no manual."

    # Montar contexto
    contexto = "\n\n".join(
        [f"[Página {m.metadata.get('pagina', 'N/A')}]\n{m.metadata.get('text', '')}" for m in matches]
    )

    # Prompt estruturado
    prompt = f"""
    Você é um assistente técnico especializado em portas industriais automáticas.
    Utilize **apenas as informações do manual abaixo** para responder à pergunta de forma precisa, técnica e completa.
    Se a resposta não estiver claramente no manual, diga que a informação não foi encontrada.

    Manual extraído:
    {contexto}

    Pergunta: {query}

    Responda:
    """

    # Gerar resposta com o modelo
    completion = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "Você responde exclusivamente com base no manual técnico fornecido."},
            {"role": "user", "content": prompt}
        ],
        temperature=0  # resposta objetiva, sem criatividade
    )

    resposta = completion.choices[0].message.content.strip()

    return resposta

In [ ]:
query = "parametros sobre freio"
response = rag_manual_chat(query, client, index)
print("Resposta:", response)


In [ ]:
# Example usage
while True:
  query = input()
  response = rag_manual_chat(query, client, index)
  print(f"User: {query}")
  print(f"Bot: {response}")